<a href="https://colab.research.google.com/github/Ryanjoo18/fires-hotspots/blob/main/Fires_and_Hotspots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fires and Hotspots

In [ ]:
#Authenticate Earth Engine
import ee
ee.Authenticate()

True

In [ ]:
ee.Initialize(project='ee-ryanjooruian18')

##Hotspots

In [ ]:
# Set the timeframe
from_date = '2023-09-01'
upto_date = '2023-10-01'

# Load FIRMS MODIS active fire dataset and filter by date
dataset = ee.ImageCollection('FIRMS').filterDate(from_date, upto_date)

# Select the confidence band (values from 0-100) and normalize to 0-1 scale
fires = dataset.select('confidence').max().divide(100)

# Mask pixels where confidence is below 90%
hotspots = fires.updateMask(fires.gte(0.9))

# Define AOI (Sumatra polygon)
aoi = ee.Geometry.Polygon([
    [[93.661, 4.92], [103.58, -7.66], [109.26, -3.25], [95.46, 6.78]]
])

# Compute total number of hotspots in AOI
stats = hotspots.rename('hotspots').reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=1000
)

# Retrieve and print the result
hotspot_count = stats.getNumber('hotspots').getInfo()
print(f"Total number of hotspots in Sumatra = {round(hotspot_count) if hotspot_count else 0}")

Total number of hotspots in Sumatra = 1638


##Terra MODIS Imagery

In [ ]:
!pip install -q earthengine-api geemap folium

In [ ]:
import geemap.core as geemap

In [ ]:
# Define the timeframe
from_date = '2023-09-28'
upto_date = '2023-09-29'

# Define the Area of Interest (AOI) - South Sumatra
aoi = ee.Geometry.Rectangle([101.99, -1.54, 106.16, -5.04])

# Load the Terra MODIS Surface Reflectance dataset
dataset = ee.ImageCollection('MODIS/061/MOD09GA').filterDate(from_date, upto_date)

# Get the first image from the collection
terra_image = dataset.first()

# Define False Color (2,1,4) visualization
false_col = terra_image.select(['sur_refl_b02', 'sur_refl_b01', 'sur_refl_b04']).clip(aoi)
false_col_vis = {
    'min': 500.0,
    'max': 4300.0
}

# Define True Color (1,4,3) visualization
true_col = terra_image.select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']).clip(aoi)
true_col_vis = {
    'min': 0.0,
    'max': 3500.0
}

# Load FIRMS dataset and filter by date and AOI
fires_dataset = ee.ImageCollection('FIRMS').filterDate(from_date, upto_date).filterBounds(aoi)

# Select confidence band and normalize it
fires = fires_dataset.select('confidence').max().divide(100)

# Mask values below 90% confidence
hotspots = fires.updateMask(fires.gte(0.9))

# Compute total number of hotspots
stats = hotspots.rename('hotspots').reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=1000
)

# Retrieve and print the result
hotspot_count = stats.getNumber('hotspots').getInfo()
print(f"Total number of hotspots in South Sumatra = {round(hotspot_count) if hotspot_count else 0}")

# Create an interactive map
map_center = [-3.40, 103.92]  # Latitude, Longitude
m = geemap.Map(center=map_center, zoom=7)

# Add layers to the map
m.addLayer(false_col, false_col_vis, 'Terra MODIS False Colour 214')
m.addLayer(true_col, true_col_vis, 'Terra MODIS True Colour')
m.addLayer(hotspots, {'palette': ['red']}, 'Hotspots')

# Display the map
m

Total number of hotspots in South Sumatra = 153


Map(center=[-3.4, 103.92], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

##Sentinel-2 Imagery

In [ ]:
# Set date range
from_date = '2023-09-29'
upto_date = '2023-09-30'

# Define AOI (South Sumatra)
AOI = ee.Geometry.Rectangle([101.99, -1.54, 106.16, -5.04])

# Load Sentinel-2 Harmonized Surface Reflectance Dataset
dataset = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
           .filterDate(from_date, upto_date)
           .filterBounds(AOI))

# Mosaic overlapping images and normalize values
S2_image = dataset.mosaic().divide(10000)

# Define visualization parameters
trueCol_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
falseCol_vis = {'min': 0.05, 'max': 0.4, 'bands': ['B8', 'B4', 'B3']}
falseColSWIR_vis = {'min': 0.05, 'max': 0.7, 'bands': ['B12', 'B8', 'B4']}

# Load FIRMS dataset (fire hotspots)
fires = (ee.ImageCollection('FIRMS')
         .filterDate(from_date, upto_date)
         .filterBounds(AOI)
         .select('confidence')
         .max()
         .divide(100))  # Normalize

# Mask pixels where confidence is below 90%
hsmask = fires.gte(0.9)
hotspots = fires.mask(hsmask)

# Initialize interactive map
Map = geemap.Map(center=[0.7, 107], zoom=6)

# Add Sentinel-2 layers
Map.addLayer(S2_image.select(['B4', 'B3', 'B2']).clip(AOI), trueCol_vis, 'Sentinel-2 True Colour')
Map.addLayer(S2_image.select(['B8', 'B4', 'B3']).clip(AOI), falseCol_vis, 'Sentinel-2 False Colour 843')
Map.addLayer(S2_image.select(['B12', 'B8', 'B4']).clip(AOI), falseColSWIR_vis, 'Sentinel-2 False Colour 12/8/4')

# Add hotspots overlay
Map.addLayer(hotspots, {'palette': ['red']}, 'Hotspots')

# Calculate number of hotspots in AOI
stats = hotspots.rename('hotspots').reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI,
    scale=1000
)
hotspot_count = stats.getNumber('hotspots').round().getInfo()
print(f"Total number of hotspots in Sumatra: {hotspot_count}")

# Display the interactive map
Map


Total number of hotspots in Sumatra: 313


Map(center=[0.7, 107], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…